# Многомерный анализ данных

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib import dates
from scipy import stats
from scipy.stats import pearsonr


## Метеоданные: Джанкуат
Скачайте данные с сайта https://doi.pangaea.de/10.1594/PANGAEA.894807.
Посмотрим на данные с метеостанции на базе гляциологов около ледника Джанкуат: Djankuat_AWS_base_daily. Не забудьте положить файл в рабочую папку.

Метеостанции на Джанкуате работают, как правило, с июня по октябрь.

Советую скачать Notepad https://notepad-plus-plus.org/downloads/, который позволяет посмотреть, как выглядит файл.
Откройте файл в Notepad и удалите первые строки с описанием данных вместе с символами /* и */.
Иначе данные прочитать проблематично.

In [30]:
#read Djankuat data
df_dj1 = pd.read_csv("Djankuat_AWS-base_daily.tab",sep='\t')

In [32]:
df_dj1.head()

,Date/Time,T2 [°C],TdTdTd [°C],RH [%],ff day m [m/s],ff max [m/s],Precip [mm/day]
0,2007-06-17,11.8,NaN,NaN,NaN,NaN,NaN
1,2007-06-18,9.8,NaN,NaN,NaN,NaN,NaN
2,2007-06-19,9.1,NaN,NaN,NaN,NaN,NaN
3,2007-06-20,6.7,NaN,NaN,NaN,NaN,NaN
4,2007-06-21,6.3,NaN,NaN,NaN,NaN,NaN


Сделайте колонку с датой индексом:

Переименуем колонки, а то неудобно.

In [ ]:
#rename columns
df_dj1.columns = ['T2','Td','RH','wind','wind_max','Prec']

### Задача: 
получить непрерывный метеоряд для условий ледника Джанкуат из имеющихся данных с метеостанции Терскол и с Джанкуата.

*Первый шаг*: 
Получить средние значения для Джанкуата с июня по октябрь.

In [33]:
df_dj1.groupby('time.year').mean()


KeyError: 'time.year'

Для этого получим средние значения для каждого дня. То есть средние многолетние значения для каждой даты: 1 августа, 2 августа и т. д. Для этого существует метод groupby. '%m-%d' - формат даты, которая получится в результате применения метода strftime.

In [37]:
djankuat_T_multiyear = df_dj1.T2.groupby(pd.to_datetime(df_dj1.index).strftime('%m-%d')).mean()

,Date/Time,T2 [°C],TdTdTd [°C],RH [%],ff day m [m/s],ff max [m/s],Precip [mm/day],year
0,2007-06-17,11.8,NaN,NaN,NaN,NaN,NaN,1970
1,2007-06-18,9.8,NaN,NaN,NaN,NaN,NaN,1970
2,2007-06-19,9.1,NaN,NaN,NaN,NaN,NaN,1970
3,2007-06-20,6.7,NaN,NaN,NaN,NaN,NaN,1970
4,2007-06-21,6.3,NaN,NaN,NaN,NaN,NaN,1970


Получилось?

In [ ]:
djankuat_T_multiyear.plot()

*Второй шаг* : сделайте то же самое для метеоданных Терскола за 2007-2018.

In [ ]:
# Загрузите метеоданные со станции Терскол, приведите их в порядок

# Выберите данные 2007 - 2018 гг.

# Сгруппируйте данные по дням года

# Выберите данные с 6 июня по 4 октября

Сравним распределения температур летнего сезона с Терскола и с Джанкуата

In [ ]:
sns.distplot(terskol_T_multiyear)
sns.distplot(djankuat_T_multiyear)

### Корреляция Пирсона
???????




Попробуем оценить, насколько коррелируют изменения температуры в Терсколе и на Джанкуате. Нас интересует, можно ли использовать данные с Терскола, чтобы воспроизвести недостающие данные на Джанкуате.

Диаграмма рассеяния - это очень распространенная и понятная визуализация количественных двумерных данных. Ниже мы делаем диаграмму рассеяния температуры на Джанкуате в зависимости от температуры в Терсколе. Это означает, что температура на Джанкуате отложена по вертикальной оси, а в Терсколе - по горизонтальной оси). Мы видим положительную зависимость между этими двумя показателями - когда теплее на Терсколе, теплее и на Джанкуате, и наоборот. 

In [ ]:
sns.regplot(x = terskol_T_multiyear, y = djankuat_T_multiyear)

Другой способ - построить график «плотности» точек. На графиках ниже более темные цвета показывают, куда попадает большее количество точек. Два поля сверху и справа от графика показывают плотности для температуры в Терсколе и на Джанкуате отдельно, в то время как график в центре показывает их плотность вместе.

Этот график также показывает коэффициент корреляции Пирсона между температурой в Терсколе и на Джанкуате, который составляет 0.91. Как обсуждалось в курсе, коэффициент корреляции Пирсона колеблется от -1 до 1, а значения, приближающиеся к 1, указывают на более совершенную положительную зависимость. Корреляцию 0.91 можно считать сильной положительной зависимостью.

In [ ]:
sns.jointplot(x=terskol_T_multiyear, y=djankuat_T_multiyear, kind='kde').annotate(stats.pearsonr)

Постройте такие графики для осадков на Джанкуате и в Терсколе. Какой получается коэффициент корреляции для осадков?

## Простая линейная регрессия

Мы можем использовать метод линейной регрессии, когда мы хотим предсказать значение одной переменной из значений одной или нескольких других переменных.
Например, когда мы ищем линейную зависимость, наиболее подходящую для набора данных $(x_i, y_i)$ (температура в Терсколе и на Джанкуате), мы ищем параметры $(k, d)$, которые минимизируют сумму квадратов остатков $\epsilon_i$ в
$$ y_i = k x_i + d + \epsilon_i$$


Можно получить необходимое уравнение для перевода температуры  в Терсколе в температуру на Джанкуате и с помощью метода библиотеки Numpy polyfit.

In [ ]:
y=djankuat_T_multiyear
x=terskol_T_multiyear
plt.scatter(x=x,y=y,label='Tmean')
plt.ylabel('Джанкуат 2640')
plt.xlabel('Терскол')
m, b = np.polyfit(x, y, 1)
plt.plot(x, m*x + b,'r',linewidth=4,label = 'Djankuat = 1.07*Terskol - 2.86')
plt.title('Линейная связь температуры на Джанкуате и в Терсколе летом 2007-2018 гг. с корреляцией 0.69')
plt.legend()
plt.show()
corr, _ = pearsonr(x,y)
print(m,b, corr)

А какое уравнение потребуется для перевода  температуры в Терсколе в температуру на леднике Джанкуат высоте 3200 м?
Используйте данные с метеостанции AWS3.